# 🔍 Conversational Question Answering with LangChain

## Overview

This notebook builds a **conversational question–answering system** using LangChain.
The system answers questions based on the content of a single web page and
maintains conversation history for follow-up queries.


In [ ]:
import os

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain



---

## What This Code Does

1. **Load Web Content**  
   Loads the Wikipedia page on *Artificial Intelligence* using a web document loader.

2. **Split Text into Chunks**  
   Splits the loaded document into smaller text chunks using a recursive character
   text splitter.

3. **Generate Embeddings**  
   Converts each text chunk into vector embeddings using OpenAI embeddings.

4. **Store Vectors in FAISS**  
   Stores the document embeddings in a FAISS vector store to enable similarity search.

5. **Enable Conversation Memory**  
   Uses a conversation buffer to keep track of previous questions and answers.

6. **Create a Conversational Retrieval Chain**  
   Combines:
   - A chat-based language model  
   - A retriever backed by the FAISS vector store  
   - Conversation memory

7. **Run a Query**  
   Submits a user question to the chain and returns an answer grounded in the
   retrieved document content.


In [ ]:
api_key = os.environ.get("OPENAI_API_KEY")  #please set your OpenAI API key in the .env file or as you prefer

In [ ]:
# Target URL
url = "https://en.wikipedia.org/wiki/Artificial_intelligence"

In [ ]:
# Load web content
loader = WebBaseLoader(url)

In [ ]:

raw_documents = loader.load()

In [ ]:
# Generate embeddings
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(raw_documents)  # Split into chunks  

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key= api_key)

In [ ]:
vectorstore = FAISS.from_documents(documents, embeddings)  #storing documents and embeddings in FAISS vector store

In [20]:
memory =  ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
qa =  ConversationalRetrievalChain.from_llm(ChatOpenAI(openai_api_key = api_key, model="gpt-5-nano", temperature=1), vectorstore.as_retriever(), memory=memory)

---

## Output

The final output is a text answer to the user’s question, generated using
retrieved context from the web page and informed by conversation history.


In [ ]:
query = "What is Artificial Intelligence?"

In [ ]:
result = qa.invoke({"question": query})

In [ ]:
result["answer"]